# foliumを使って色んな地図を重ねて表示する

In [60]:
import folium
from folium.plugins import LocateControl

center = [35.010888, 135.759509] # 初期地図の設定（京都市付近）
m = folium.Map(location=center,
               #tiles=None, # base mapを非表示
               zoom_start=13, max_zoom=20, min_zoom=13,control_scale=True)
               #width=600)
#m.get_root().height = 600 # 縦サイズを指定

# ベースマップ(tiles=Noneを変えた場合)
#folium.TileLayer('openstreetmap', name='OpenStreetMap').add_to(m)
# オーバーレイ(非表示)
overlay_layers =[
# 国土地理院 https://maps.gsi.go.jp/development/ichiran.html#ort_riku10
{'name':'全国最新写真(シームレス)',"attr":'国土地理院+GRUS画像（© Axelspace）',
'tiles':'https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg'},
{'name':'国土地理院 陰影起伏図',"attr":'国土地理院',
'tiles':'https://cyberjapandata.gsi.go.jp/xyz/hillshademap/{z}/{x}/{y}.png'},
{'name':'国土地理院 空中写真（1945年～1950年）',"attr":'国土地理院',
'tiles':'https://cyberjapandata.gsi.go.jp/xyz/ort_USA10/{z}/{x}/{y}.png'},
{'name':'国土地理院 空中写真（1936年～1942年頃 東京大阪）',"attr":'国土地理院',
'tiles':'https://cyberjapandata.gsi.go.jp/xyz/ort_USA10/{z}/{x}/{y}.png'}
]
for layer in overlay_layers:
    folium.TileLayer(attr=layer['attr'],name=layer['name'],tiles=layer['tiles'],
        opacity=0.75,overlay=True,show=False).add_to(m)
# 立命館大学提供のレイヤーをオーバーレイとしてfoliumマップに追加
ritsumei_layers = {
    "meisai_sougou": "昭和26年頃京都市明細図(総合資料館版)【around 1951】",
    "meisai_hasegawa": "昭和2年頃京都市明細図(長谷川家版)【around 1926】",
    "s28": "昭和28年(都市計画)【1953】",
    "s10": "昭和10年(都市計画)【1935】",
    "s4": "昭和4年(都市計画)【1929】",
    "mono_s4": "昭和4年mono(都市計画)【1929】",
    "mono_t11": "大正11年mono(都市計画)【1922】",
    "seisiki": "大正元年(正式図)【1912】",
    "kasei": "明治25年(仮製図)【1892】"}
for layer_key, layer_name in ritsumei_layers.items():
    url_template = f"https://www.arc.ritsumei.ac.jp/archive01/theater/image/PB/geo/maps/{layer_key}/{{z}}/{{x}}/{{y}}.png"
    folium.raster_layers.TileLayer(
        tiles=url_template,
        attr="近代京都オーバーレイマップ",
        name=layer_name,
        overlay=True, # baseレイヤーでなくoverlayレイヤーとする
        show=False, # 初期は非表示
        opacity=0.75,  # 初期透明度
        tms=True  # TMS形式を指定
    ).add_to(m)
# オーバーレイ(非表示)
# 今昔マップタイル https://ktgis.net/kjmapw/tilemapservice.html
overlay_layers =[
{'name':'今昔マップ 首都圏 1896-1909年',"attr":'今昔マップ on the web',
'tiles':'https://ktgis.net/kjmapw/kjtilemap/tokyo50/2man/{z}/{x}/{y}.png'},
{'name':'今昔マップ 京阪神圏 1892-1910年',"attr":'今昔マップ on the web',
'tiles':'https://ktgis.net/kjmapw/kjtilemap/keihansin/2man/{z}/{x}/{y}.png'}]
for layer in overlay_layers:
    folium.TileLayer(attr=layer['attr'],name=layer['name'],tiles=layer['tiles'],
        overlay=True,show=False,tms=True).add_to(m)
    
folium.LayerControl().add_to(m) # レイヤーコントロールを追加
folium.plugins.LocateControl(auto_start=False).add_to(m) # デバイス位置表示オプション追加
m.save("_map.html")# 一旦、htmlファイルとして保存する
#m #display(m) # Jupyter Notebook で表示

tile_names_id = [] # overlayタイルを格納する
for key, value in m._children.items():
    if isinstance(value, folium.TileLayer) and value.overlay:
        #print(vars(value))
        tile_names_id.append(f'tile_layer_{value._id}')

with open("_map.html", 'r') as f: # HTMLファイルを読み込みHTMLコードを挿入する
    html_content = f.read()
custom_html = '''
<div style="position:absolute;bottom:20px;right:10px;z-index:1000;background:rgba(255,255,255,0.9);padding:10px;">
<input id="slide" type="range" min="0" max="1" step="0.1" value="1" onchange="updateOpacity(this.value)"></div>'''
html_content = html_content.replace('</body>', f'{custom_html}</body>')# </body>直前に挿入
custom_html = '<script>function updateOpacity(value) {'+\
''.join([f"{name_id}.setOpacity(value);" for name_id in tile_names_id])+"}</script>"
html_content = html_content.replace('</html>', f'{custom_html}</html>')# </html>直前に挿入
with open("map.html", 'w') as f:# 追記コードを挿入したHTMLを保存する
    f.write(html_content)

In [61]:
from IPython.display import HTML

HTML('"<iframe width="600" height="600" src="./map.html"></iframe>"')